<a href="https://colab.research.google.com/github/Bhaviya22/Deep_learning-Projects/blob/main/langchain_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain llama-cpp-python faiss-cpu sentence-transformers #-q=quite running or downloading

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 MB 17.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!wget -q https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf -O tinyllama.gguf #!wget->directly upload the model from online sources or drive


In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import TextLoader #to get the doc
from langchain_text_splitters import CharacterTextSplitter #to seprate as chunks before embeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings # embeddings
from langchain.vectorstores import FAISS #vector db
from langchain.chains import RetrievalQA # acts as a chains
from langchain.llms import LlamaCpp # to run model in our cpu

In [ ]:
loader=TextLoader("text.txt")
documents=loader.load()

splitter=CharacterTextSplitter(chunk_size=200,chunk_overlap=50)
docs=splitter.split_documents(documents)

embeddings=HuggingFaceBgeEmbeddings(model_name="all-MiniLM-L6-v2")
vectorestore=FAISS.from_documents(docs,embeddings)

In [ ]:
llm=LlamaCpp(model_path="tinyllama.gguf",
             temperature=0.5,# to think and give creative
             max_tokens=512 ,#max tokens allowd
             top_p=0.95 ,#to give meaningful text

             #verbose=True)    #to see the details shown bellow
)

llama_model_loader: loaded meta data with 23 key-value pairs and 201 tensors from tinyllama.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = tinyllama_tinyllama-1.1b-chat-v1.0
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 22
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5632
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 64
llama_model_loader: - kv   7:                 llama.attention.head_count

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorestore.as_retriever()
)
question = "explain about lithosphere?"  # ✅ No comma here

# ✅ Fixed method name: should be qa_chain.run(), not qa_chainrun
result = qa_chain.run(question)

print("Answer:", result)

llama_perf_context_print:        load time =   11668.22 ms
llama_perf_context_print: prompt eval time =   11668.00 ms /   213 tokens (   54.78 ms per token,    18.26 tokens per second)
llama_perf_context_print:        eval time =   11635.15 ms /   112 runs   (  103.89 ms per token,     9.63 tokens per second)
llama_perf_context_print:       total time =   23444.63 ms /   325 tokens


Answer:  The lithosphere is a layer of the Earth's crust that consists primarily of rock and minerals. It sits atop the asthenosphere, which is made up mostly of molten rock.
The asthenosphere lies below the lithosphere and has a texture like plastic. The increased temperature and pressure make rocks in this layer more molten than those above.
Asnesspheric material can also rise to the surface when plate tectonics occur, forming and recycling lithosphere material.
